In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import KFold, RepeatedKFold
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [37]:
# df_train=pd.read_csv("E:\\jinnan\\train_after_clean.csv")
df_train=pd.read_csv("E:\\jinnan\\train_dispersion.csv")
df_train=df_train[df_train['yield_rate']>0.87]
df_train.drop(['B3', 'B13', 'A13', 'A18', 'A23'], axis=1, inplace=True)
for col in df_train.columns:
    rate = df_train[col].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.9:
        df_train.drop(columns=[col], axis=1, inplace=True)
label_array=df_train["yield_rate"].values
df_train["id"]=df_train['id'].apply(lambda x: int(x[7:]))
df_train.drop(columns=['yield_rate'],axis=1,inplace=True)
print('train1:',df_train.shape)
df_train2=pd.read_csv("E:\\jinnan\\train_binning_mean.csv")
print('train2:',df_train2.shape)
df_train.index=range(df_train.shape[0])
df_train2.index=range(df_train2.shape[0])
df_train=pd.concat([df_train,df_train2],axis=1)
train_array=df_train.values
print(df_train.columns,df_train.shape)

# df_test=pd.read_csv("E:\\jinnan\\test_after_clean.csv")
df_test=pd.read_csv("E:\\jinnan\\test_dispersion.csv")
df_test.drop(['B3', 'B13', 'A13', 'A18', 'A23'], axis=1, inplace=True)
for col in df_test.columns:
    rate = df_test[col].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.9:
        df_test.drop(columns=[col], axis=1, inplace=True)
columns_list=list(df_test.columns)
# columns_list.remove("id")
sample_id=df_test["id"]
df_test["id"]=df_test['id'].apply(lambda x: int(x[7:]))

df_test2=pd.read_csv("E:\\jinnan\\testA_binning_mean.csv")
df_test=pd.concat([df_test,df_test2],axis=1)
test_array=df_test.values
print(df_test.columns,df_test.shape)

train1: (1381, 78)
train2: (1381, 105)
Index(['A10', 'A11', 'A12', 'A14', 'A15', 'A16', 'A17', 'A19', 'A20',
       'A20range',
       ...
       'B14_to_B11_intTarget_0.0_mean', 'B14_to_B11_intTarget_1.0_mean',
       'B14_to_B11_intTarget_2.0_mean', 'B14_to_B11_intTarget_3.0_mean',
       'B14_to_B11_intTarget_4.0_mean', 'B14_to_B14_intTarget_0.0_mean',
       'B14_to_B14_intTarget_1.0_mean', 'B14_to_B14_intTarget_2.0_mean',
       'B14_to_B14_intTarget_3.0_mean', 'B14_to_B14_intTarget_4.0_mean'],
      dtype='object', length=183) (1381, 183)
Index(['A10', 'A11', 'A12', 'A14', 'A15', 'A16', 'A17', 'A19', 'A20',
       'A20range',
       ...
       'B14_to_B11_intTarget_0.0_mean', 'B14_to_B11_intTarget_1.0_mean',
       'B14_to_B11_intTarget_2.0_mean', 'B14_to_B11_intTarget_3.0_mean',
       'B14_to_B11_intTarget_4.0_mean', 'B14_to_B14_intTarget_0.0_mean',
       'B14_to_B14_intTarget_1.0_mean', 'B14_to_B14_intTarget_2.0_mean',
       'B14_to_B14_intTarget_3.0_mean', 'B14_to_B14_intTa

In [38]:
param = {'num_leaves': 120,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 30,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'mse',
         "lambda_l1": 0.1,
         "verbosity": -1}
folds = KFold(n_splits=5, shuffle=True, random_state=2018)
oof_lgb = np.zeros(train_array.shape[0])
predictions_lgb = np.zeros(test_array.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_array, label_array)):
    print("fold n°{}".format(fold_+1))
    trn_data = lgb.Dataset(train_array[trn_idx], label_array[trn_idx])
    val_data = lgb.Dataset(train_array[val_idx], label_array[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 100)
    oof_lgb[val_idx] = clf.predict(train_array[val_idx], num_iteration=clf.best_iteration)
    
    predictions_lgb += clf.predict(test_array, num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.8f}".format(mean_squared_error(oof_lgb, label_array)))

fold n°1
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000195907	valid_1's l2: 0.000222516
[400]	training's l2: 0.000122574	valid_1's l2: 0.000151465
[600]	training's l2: 0.00010185	valid_1's l2: 0.000133858
[800]	training's l2: 9.26023e-05	valid_1's l2: 0.000126702
[1000]	training's l2: 8.6976e-05	valid_1's l2: 0.000123004
[1200]	training's l2: 8.32183e-05	valid_1's l2: 0.000120659
[1400]	training's l2: 8.03884e-05	valid_1's l2: 0.000119188
[1600]	training's l2: 7.80774e-05	valid_1's l2: 0.000117972
[1800]	training's l2: 7.62917e-05	valid_1's l2: 0.000117192
[2000]	training's l2: 7.47053e-05	valid_1's l2: 0.000116554
[2200]	training's l2: 7.33105e-05	valid_1's l2: 0.000116036
Early stopping, best iteration is:
[2279]	training's l2: 7.28173e-05	valid_1's l2: 0.000115809
fold n°2
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000188351	valid_1's l2: 0.000209668
[400]	training's l2: 0.000116997	valid_1's l2: 0

In [39]:
xgb_params = {'eta': 0.005, 'max_depth': 10, 'subsample': 0.8, 'colsample_bytree': 0.8,
          'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 4}
folds = KFold(n_splits=5, shuffle=True, random_state=2018)
oof_xgb = np.zeros(len(df_train))
predictions_xgb = np.zeros(len(df_test))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_array, label_array)):
    print("fold n°{}".format(fold_ + 1))
    trn_data = xgb.DMatrix(train_array[trn_idx], label_array[trn_idx])
    val_data = xgb.DMatrix(train_array[val_idx], label_array[val_idx])

    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200,
                    verbose_eval=100, params=xgb_params)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(train_array[val_idx]), ntree_limit=clf.best_ntree_limit)
    predictions_xgb += clf.predict(xgb.DMatrix(test_array), ntree_limit=clf.best_ntree_limit) / folds.n_splits
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, label_array)))

fold n°1
[0]	train-rmse:0.422932	valid_data-rmse:0.423819
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[100]	train-rmse:0.257001	valid_data-rmse:0.25773
[200]	train-rmse:0.156615	valid_data-rmse:0.157312
[300]	train-rmse:0.095902	valid_data-rmse:0.096701
[400]	train-rmse:0.059306	valid_data-rmse:0.060168
[500]	train-rmse:0.037293	valid_data-rmse:0.038285
[600]	train-rmse:0.024087	valid_data-rmse:0.025363
[700]	train-rmse:0.016187	valid_data-rmse:0.018069
[800]	train-rmse:0.011512	valid_data-rmse:0.014177
[900]	train-rmse:0.008755	valid_data-rmse:0.012217
[1000]	train-rmse:0.007123	valid_data-rmse:0.011297
[1100]	train-rmse:0.006152	valid_data-rmse:0.010879
[1200]	train-rmse:0.005518	valid_data-rmse:0.010705
[1300]	train-rmse:0.005053	valid_data-rmse:0.010621
[1400]	train-rmse:0.004687	valid_data-rmse:0.010586
[1500]	train-rmse:0.004363	valid_data-rmse:0.010583
[1600]	train-rms

In [41]:
print("mean_lgb:",np.mean(predictions_lgb))
print("var_lgb:",np.var(predictions_lgb))
print("mean_xgb:",np.mean(predictions_xgb))
print("var_xgb:",np.var(predictions_xgb))

mean_lgb: 0.925775245467878
var_lgb: 0.0006136959920948989
mean_xgb: 0.9249380680918694
var_xgb: 0.0006400234054478957


In [27]:
df_predicted=pd.DataFrame({"id":sample_id.values,"predicted":predictions_lgb})
df_predicted.loc[:,"predicted"]=df_predicted["predicted"].apply(lambda x: round(x,3))
df_predicted.to_csv("E:\\jinnan\\predict_result\\jinnan_idfea_binning_dispersion_lgb_1_11.csv",header=None,index=False)
df_predicted

,id,predicted
0,sample_1656,0.904
1,sample_1548,0.880
2,sample_769,0.936
3,sample_1881,0.905
4,sample_1807,0.936
5,sample_145,0.923
6,sample_1212,0.929
7,sample_944,0.899
8,sample_829,0.934
9,sample_616,0.923


In [29]:
df_predicted=pd.DataFrame({"id":sample_id.values,"predicted":predictions_xgb})
df_predicted.loc[:,"predicted"]=df_predicted["predicted"].apply(lambda x: round(x,3))
df_predicted.to_csv("E:\\jinnan\\predict_result\\jinnan_idfea_binning_dispersion_xgb_1_11.csv",header=None,index=False)
df_predicted

,id,predicted
0,sample_1656,0.906
1,sample_1548,0.879
2,sample_769,0.933
3,sample_1881,0.905
4,sample_1807,0.928
5,sample_145,0.926
6,sample_1212,0.932
7,sample_944,0.900
8,sample_829,0.933
9,sample_616,0.928


In [23]:
from sklearn.linear_model import BayesianRidge

In [42]:
train_stack = np.vstack([oof_lgb,oof_xgb]).transpose()
test_stack = np.vstack([predictions_lgb, predictions_xgb]).transpose()

folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=4590)
oof_stack = np.zeros(train_stack.shape[0])
predictions = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack,label_array)):
    print("fold {}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], label_array[trn_idx]
    val_data, val_y = train_stack[val_idx], label_array[val_idx]
    
    clf_3 = BayesianRidge()
    clf_3.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf_3.predict(val_data)
    predictions += clf_3.predict(test_stack) / 10
    
mean_squared_error(label_array, oof_stack)

fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9


0.00011789490765092857

In [43]:
print("mean:",np.mean(predictions))
print("var:",np.var(predictions))

mean: 0.9255877522086162
var: 0.000645417178952132


In [28]:
df_predicted=pd.DataFrame({"id":sample_id.values,"predicted":predictions})
df_predicted.loc[:,"predicted"]=df_predicted["predicted"].apply(lambda x: round(x,3))
df_predicted.to_csv("E:\\jinnan\\predict_result\\jinnan_idfea_binning_dispersion_stacking_1_11.csv",header=None,index=False)
df_predicted

,id,predicted
0,sample_1656,0.905
1,sample_1548,0.879
2,sample_769,0.934
3,sample_1881,0.905
4,sample_1807,0.932
5,sample_145,0.925
6,sample_1212,0.931
7,sample_944,0.899
8,sample_829,0.934
9,sample_616,0.926
